In [1]:
import requests
from langchain_openai import ChatOpenAI
import os
import json
import re
from collections import defaultdict

os.environ["OPENAI_API_KEY"] = "sk-j5nZtClYNnChRcfn85757c9bC69140BfBc9f3aF5A37f046a"
os.environ["OPENAI_API_BASE"] = "https://api.mixrai.com/v1"
llm = ChatOpenAI(model="gpt-4o")

In [2]:
url = "https://raw.githubusercontent.com/dwu5/NLP_Proj/refs/heads/main/data/2025%E5%9B%BD%E8%80%83%E8%A1%8C%E6%B5%8B_94%E9%A2%98.json"
response = requests.get(url)
data = response.json()
questions = data["questions"]
skip_ids = set(map(str, data["skip"]))

In [3]:
file_name = data["title"]
print(file_name)

type_counts = defaultdict(int)
for question in questions:
    if question["id"] not in skip_ids:
        type_counts[question["type"]] += 1

print("\n各题型题数统计 (剔除skip题):")
for type_name, count in type_counts.items():
    print(f"{type_name}: {count}题")
print(f"\n总题数 (剔除skip题): {sum(type_counts.values())}")
print(f"跳过的题数: {len(skip_ids)}")

2025年国家公务员录用考试《行测》题（副省级网友回忆版）

各题型题数统计 (剔除skip题):
政治理论: 20题
常识判断: 15题
表达理解: 20题
数量关系: 14题
判断推理: 24题

总题数 (剔除skip题): 93
跳过的题数: 42


In [4]:
def predict_answer(question_data):
    # 构建完整题目文本（包含选项）
    question_text = question_data["question"]
    if question_data["options"]:  # 有选项
        options_text = "\n".join([f"{k}、{v}" for k, v in question_data["options"].items()])
        full_question = f"{question_text}\n{options_text}"
    else:
        full_question = question_text  # 没有选项
    
    prompt = f"""
    请根据题目内容选择最合适的答案。只需返回选项字母，不要包含其他任何内容！

    题目：
    {full_question}
    """
    response = llm.invoke(prompt)
    # 提取选项字母
    pred_answer = response.content.strip()
    match = re.search(r'([A-D]+)', pred_answer.upper())
    if match:
        return match.group(1)
    return pred_answer

In [5]:
results = []
correct = 0
total_processed = 0  # 实际处理的题目数（剔除skip题）

# 各题型准确率统计
type_stats = defaultdict(lambda: {'correct': 0, 'total': 0})

for q in questions:
    # 跳过skip列表中的题目
    if q["id"] in skip_ids or "answer" not in q:
        continue 
    
    total_processed += 1
    
    try:
        pred_answer = predict_answer(q)
        true_answer = q["answer"]
        is_correct = pred_answer == true_answer
        
        # 更新总体统计
        if is_correct:
            correct += 1
        
        # 更新题型统计
        question_type = q["type"]
        type_stats[question_type]['total'] += 1
        if is_correct:
            type_stats[question_type]['correct'] += 1
        
        # 记录结果
        results.append({
            "id": q["id"],
            "type": q["type"],
            "question": q["question"],
            "options": q["options"],
            "true_answer": true_answer,
            "pred_answer": pred_answer,
            "is_correct": is_correct
        })
            
    except Exception as e:
        print(f"Error processing question {q['id']}: {e}")
        # 出错记录
        question_type = q["type"]
        type_stats[question_type]['total'] += 1
        
        results.append({
            "id": q["id"],
            "type": q["type"],
            "question": q["question"],
            "options": q["options"],
            "true_answer": q.get("answer", ""),
            "pred_answer": "ERROR",
            "is_correct": False,
            "error": str(e)
        })

# 计算准确率（基于实际处理的题目）
accuracy = correct / total_processed if total_processed > 0 else 0

# 计算各题型准确率
type_accuracies = {}
for type_name, stats in type_stats.items():
    type_total = stats['total']
    type_correct = stats['correct']
    type_accuracies[type_name] = {
        'accuracy': type_correct / type_total if type_total > 0 else 0,
        'correct': type_correct,
        'total': type_total
    }

print(f"\n总体准确率(剔除skip题目): {accuracy:.2%} ({correct}/{total_processed})")
print("\n各题型准确率:")
for type_name, acc in type_accuracies.items():
    print(f"{type_name}: {acc['accuracy']:.2%} ({acc['correct']}/{acc['total']})")


总体准确率(剔除skip题目): 78.49% (73/93)

各题型准确率:
政治理论: 90.00% (18/20)
常识判断: 86.67% (13/15)
表达理解: 90.00% (18/20)
数量关系: 42.86% (6/14)
判断推理: 75.00% (18/24)


In [6]:
# 保存结果
output_file = f"results/gpt-4o/{file_name}.json"
# os.makedirs("results", exist_ok=True)

with open(output_file, "w", encoding="utf-8") as f:
    json.dump({
        "title": data["title"],
        "skip": data["skip"],
        "overall_accuracy": accuracy,
        "total_processed_questions": total_processed,
        "correct_answers": correct,
        "skipped_questions": len(skip_ids),
        "type_accuracies": type_accuracies,
        "results": results
    }, f, ensure_ascii=False, indent=2)

print(f"预测结果已保存到 {output_file}")

预测结果已保存到 results/gpt-4o/2025年国家公务员录用考试《行测》题（副省级网友回忆版）.json
